In [1]:
from model import *
my_model = mymodel().cuda()
my_model.eval()
my_model.load_state_dict(torch.load('/data/private/E2E/gen_model/repro/try_1/6/model'))
print('ok') 

ok


In [ ]:
['<name>', '<eatType>', '<priceRange>', '<customer rating>', '<near>', '<food>', '<area>', '<familyFriendly>']

In [2]:
import random
import pandas as pd
csv_file='dataset/testset_w_refs.csv'
dataset = pd.read_csv(csv_file)
columns = dataset.columns
conditions = dataset[columns[0]]
        
typ_list = {}
for k in range(len(conditions)):
    cond_set = conditions[k].split(',')
    for m in range(len(cond_set)):
        cond_set[m] = cond_set[m].strip()
        pos = cond_set[m].index('[')
        if cond_set[m][:pos] in typ_list.keys():
            typ_list[cond_set[m][:pos]].add(cond_set[m][pos+1:-1])
        else:            
            typ_list[cond_set[m][:pos]] = {cond_set[m][pos+1:-1]}   

def sample_batch(tokenizer, cond_name, cond_set):
    condition_string = ''
    for m in range(len(cond_set)):
        condition_string += cond_name[m] + cond_set[m] + ' '

    input_string = condition_string + '<START>'
    input_ids = torch.tensor(tokenizer.encode(input_string, add_special_tokens=True))

    input_len = len(input_ids)

    return input_ids, condition_string

In [41]:
from bert_score import BERTScorer
scorer = BERTScorer(lang="en",  rescale_with_baseline=False)

the_name_list = []
name_list = []
for name in typ_list['name']:
    if 'the' in name.lower():
        the_name_list.append(name)
    else:
        name_list.append(name)
        
the_near_list = []
near_list = []
for near in typ_list['near']:
    if 'the' in near.lower():
        the_near_list.append(near)
    else:
        near_list.append(near)                

In [102]:
def find_cand(value, typ_list):
    best_score = 0 
    for v in typ_list:
        score = scorer.score([value.lower()], [v.lower()])[2].item()
        if score > best_score:
            best_score = score
            placeholder_value = v
    return placeholder_value

In [116]:
# <name> <eatType> <food> <priceRange> <customer rating> <area> <familyFriendly> <near>
random.seed(10)

name = 'Blue Man' # None
eatType = None # None
food = 'hot food' # None 
priceRange = None # '<priceRange>'
customer_rating = '2.1 out of 5' # '<CUSTOMER_RATING>' # None 
area = 'countryside' # None
familyFriendly = None
near = 'the school' # None

# name = 'The Guy' # None
# eatType = None # None
# food = 'German' # None 
# priceRange = 'expensive' # '<priceRange>'
# customer_rating = None # '<CUSTOMER_RATING>' # None 
# area = None # 'countryside' # None
# familyFriendly = 'yes' # None
# near = None #'the school' # None

cond_list = []
unseen_list = []
conditions = []
if name is not None:
    if 'the' in name.lower():
        placeholder_name = find_cand(name, the_name_list)
    else:
        placeholder_name = find_cand(name, name_list)
    cond_list.append('<name>')
    unseen_list.append(name)
    conditions.append(placeholder_name)
if eatType is not None:
    placeholder_eatType = find_cand(eatType, typ_list['eatType'])    
    cond_list.append('<eatType>')
    unseen_list.append(eatType)
    conditions.append(placeholder_eatType)
if food is not None:
    placeholder_food = find_cand(food, typ_list['food'])
    cond_list.append('<food>')
    unseen_list.append(food)
    conditions.append(placeholder_food)
if priceRange is not None:
    placeholder_priceRange = find_cand(priceRange, typ_list['priceRange'])
    cond_list.append('<priceRange>')
    unseen_list.append(priceRange)
    conditions.append(placeholder_priceRange)    
if customer_rating is not None:
    placeholder_customer_rating = find_cand(customer_rating, typ_list['customer rating'])
    cond_list.append('<customer rating>')
    unseen_list.append(customer_rating)
    conditions.append(placeholder_customer_rating)        
if area is not None:
    placeholder_area = find_cand(area, typ_list['area'])
    cond_list.append('<area>')
    unseen_list.append(area)
    conditions.append(placeholder_area)    
if familyFriendly is not None:
    cond_list.append('<familyFriendly>')
    unseen_list.append(familyFriendly)
    conditions.append(familyFriendly)            
if near is not None:
    if 'the' in near.lower():
        placeholder_near = find_cand(near, the_near_list)
    else:
        placeholder_near = find_cand(near, scorer)
    cond_list.append('<near>')
    unseen_list.append(near)
    conditions.append(placeholder_near)        


# del cond_name[2]
# del conditions[2]

sample = sample_batch(my_model.tokenizer, cond_list, conditions)

input_ids = sample[0].cuda()
condition_string = sample[1]  
input_len = len(input_ids)

max_len = 70
for _ in range(max_len):
    model_out = my_model.model_feeding(input_ids) # (batch, seq_len, emb_dim)
    pred_idx = model_out.argmax(1)[-1]        
    if pred_idx == my_model.tokenizer.eos_token_id:
        break            
    input_ids = torch.cat((input_ids, pred_idx.unsqueeze(0)), 0)        

out_sen = my_model.tokenizer.decode(input_ids[input_len:])

for cond in cond_list:
    print(cond, end=', ')
print('')

for unseen in unseen_list:
    print(unseen, end=', ')
print('')

for condition in conditions:
    print(condition, end=', ')
print('')

print(out_sen)

if name is not None:
    out_sen = out_sen.replace(placeholder_name, name)
if eatType is not None:
    out_sen = out_sen.replace(placeholder_eatType, eatType)
if food is not None:
    out_sen = out_sen.replace(placeholder_food, food)
if priceRange is not None:
    out_sen = out_sen.replace(placeholder_priceRange, priceRange)
if customer_rating is not None:
    out_sen = out_sen.replace(placeholder_customer_rating, customer_rating)
if area is not None:
    out_sen = out_sen.replace(placeholder_area, area)  
if near is not None:
    out_sen = out_sen.replace(placeholder_near, near)
print(out_sen)

<name>, <food>, <customer rating>, <area>, <near>, 
Blue Man, hot food, 2.1 out of 5, countryside, the school, 
Green Man, Fast food, 3 out of 5, city centre, The Bakers, 
Green Man is a fast food restaurant in the city centre near The Bakers. It has a customer rating of 3 out of 5.
Blue Man is a fast food restaurant in the countryside near the school. It has a customer rating of 2.1 out of 5.


In [51]:
placeholder_name

'Ranch'

In [29]:
for k, v in typ_list.items():
    print(k, v)

name {'Green Man', 'The Mill', 'The Plough', 'The Vaults', 'Blue Spice', 'Zizzi', 'The Cricketers', 'Cotto', 'The Punter', 'Strada', 'Cocum', 'The Waterman', 'The Wrestlers', 'Giraffe', 'Loch Fyne', 'The Phoenix', 'Wildwood', 'Clowns'}
eatType {'pub', 'coffee shop', 'restaurant'}
area {'city centre', 'riverside'}
customer rating {'high', '3 out of 5', '5 out of 5', 'average', 'low', '1 out of 5'}
near {'Crowne Plaza Hotel', 'Express by Holiday Inn', 'Raja Indian Cuisine', 'Burger King', 'The Rice Boat', 'Ranch', 'The Sorrento', 'Café Sicilia', 'The Bakers', 'Rainbow Vegetarian Café', 'All Bar One', 'Avalon', 'Café Rouge'}
food {'English', 'Italian', 'Japanese', 'French', 'Fast food', 'Chinese', 'Indian'}
familyFriendly {'no', 'yes'}
priceRange {'less than £20', 'high', 'cheap', 'moderate', '£20-25', 'more than £30'}


In [91]:
best_score = 0 
for v in typ_list['name']:
    score = scorer.score([name], [v])[2].item()
    print(v, score)
    if score > best_score:
        best_score = score
        placeholder_name = v
#         print(placeholder_name)

Green Man 0.8759932518005371
The Mill 0.8806518912315369
The Plough 0.8600329756736755
The Vaults 0.8975980281829834
Blue Spice 0.8518051505088806
Zizzi 0.8148564696311951
The Cricketers 0.8490865230560303
Cotto 0.8604665994644165
The Punter 0.8699408769607544
Strada 0.8372035026550293
Cocum 0.8478741645812988
The Waterman 0.8394536375999451
The Wrestlers 0.8739649653434753
Giraffe 0.858993411064148
Loch Fyne 0.8448023200035095
The Phoenix 0.8783962726593018
Wildwood 0.8554384112358093
Clowns 0.8750399947166443


In [89]:
name

'The cold'